In [ ]:
def convert_jet_to_grey(img):
    (height, width) = img.shape[:2]

    cm = LinearSegmentedColormap("turbo", _jet_data, N=2 ** 8)
    # cm = colormaps['turbo'] swap with jet if you use turbo colormap instead
    cm = colormaps['turbo']

    cm._init()  # Must be called first. cm._lut data field created here

    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    fm = cv2.FlannBasedMatcher(index_params, search_params)

    # JET, BGR order, excluding special palette values (>= 256)
    fm.add(255 * np.float32([cm._lut[:256, (2, 1, 0)]]))  # jet
    fm.train()

    # look up all pixels
    query = img.reshape((-1, 3)).astype(np.float32)
    matches = fm.match(query)

    # statistics: `result` is palette indices ("grayscale image")
    output = np.uint16([m.trainIdx for m in matches]).reshape(height, width)
    result = np.where(output < 256, output, 0).astype(np.uint8)
    # dist = np.uint8([m.distance for m in matches]).reshape(height, width)

    return result  # , dist uncomment if you wish accuracy image

def process_image(image_2d, crop_size, last_coords, threshold_mean_intensity=30):
    blurred_image = cv2.GaussianBlur(image_2d, (9, 9), 0)
    crop_half = crop_size // 2

    if last_coords is not None:
        # Define the size of the search box around the last coordinates
        search_box_size = 30
        row_center, col_center = last_coords
        row_start = max(0, row_center - search_box_size)
        row_end = min(image_2d.shape[0], row_center + search_box_size)
        col_start = max(0, col_center - search_box_size)
        col_end = min(image_2d.shape[1], col_center + search_box_size)

        # Extract the sub-image around the last coordinates
        temp_frame = blurred_image[row_start:row_end, col_start:col_end]

        # Find the highest intensity pixel in the sub-image
        max_intensity = np.max(temp_frame)
        max_intensity_index = np.argmax(temp_frame)
        row_sub_center, col_sub_center = np.unravel_index(max_intensity_index, temp_frame.shape)

        # Convert sub-image coordinates to original image coordinates
        row_center = row_start + row_sub_center
        col_center = col_start + col_sub_center

        # Determine the cropping boundaries
        row_start = max(0, row_center - crop_half)
        row_end = min(image_2d.shape[0], row_center + crop_half)
        col_start = max(0, col_center - crop_half)
        col_end = min(image_2d.shape[1], col_center + crop_half)

        # Ensure the crop size is 100x100
        if row_end - row_start < crop_size:
            if row_start == 0:
                row_end = min(crop_size, image_2d.shape[0])
            else:
                row_start = max(0, row_end - crop_size)
        if col_end - col_start < crop_size:
            if col_start == 0:
                col_end = min(crop_size, image_2d.shape[1])
            else:
                col_start = max(0, col_end - crop_size)

        # Crop the image
        cropped_image = image_2d[row_start:row_end, col_start:col_end]

        # Check the mean intensity of the cropped image
        mean_intensity = np.mean(cropped_image)
    if mean_intensity < threshold_mean_intensity or last_coords==None:
        # If last_coords is None, perform a global search
        max_intensity = np.max(blurred_image)
        max_intensity_index = np.argmax(blurred_image)
        row_center, col_center = np.unravel_index(max_intensity_index, image_2d.shape)

        # Recalculate the cropping boundaries
        row_start = max(0, row_center - crop_half)
        row_end = min(image_2d.shape[0], row_center + crop_half)
        col_start = max(0, col_center - crop_half)
        col_end = min(image_2d.shape[1], col_center + crop_half)

        # Ensure the crop size is 100x100
        if row_end - row_start < crop_size:
            if row_start == 0:
                row_end = min(crop_size, image_2d.shape[0])
            else:
                row_start = max(0, row_end - crop_size)
        if col_end - col_start < crop_size:
            if col_start == 0:
                col_end = min(crop_size, image_2d.shape[1])
            else:
                col_start = max(0, col_end - crop_size)

        # Crop the image
        cropped_image = image_2d[row_start:row_end, col_start:col_end]
        #plt.imshow(cropped_image)
        #plt.show()
        #plt.imshow(image_2d)
        #plt.show()

    # Return the coordinates of the max intensity pixel
    return [row_center, col_center], cropped_image

processed = []

for dat in data_array:
    dat = dat[..., ::-1]
    dat = convert_jet_to_grey(dat)
    last_coords = None
    if np.mean(dat)<20:
      if last_coords is None:
          max_intensity_index = np.argmax(dat)
          last_coords = np.unravel_index(max_intensity_index, dat.shape)

      # Process the image and get the coordinates of the highest intensity pixel
      current_coords, ima = process_image(dat, 50, last_coords)

      processed.append(ima)
      last_coords = current_coords